In [1]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.tools import tool
from langchain_ollama import OllamaLLM
from langchain_chroma import Chroma
from langchain_community.tools import DuckDuckGoSearchResults, YouTubeSearchTool, AzureAiServicesDocumentIntelligenceTool, WikipediaQueryRun

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict
import os
import datetime

/home/sam/Github/365DaysOfData/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = OllamaLLM(model= 'llama3.2')

llm.invoke("Hello, Ollama!")

"Hello! It's nice to meet you. I'm here to help answer any questions or chat with you about any topic that interests you. How's your day going so far?"

# 1. contextual q&a with memory
build a chatbot that not only answers questions about a dataset (like pdfs or notes) but also remembers past queries using langgraph’s state nodes.
bonus: add a “forget” command that clears memory.

In [3]:
from langchain_community.document_loaders import PyPDFLoader

doc = PyPDFLoader("/home/sam/Github/365DaysOfData/14-Agentic-AI/code/extras/geeta.pdf")
docs = doc.load()
parser = StrOutputParser()



In [4]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large")
vectorstore = Chroma.from_documents(docs, embedding=embeddings)

### Using Langchain

In [5]:
from langchain_core.prompts import ChatPromptTemplate


chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Bhagavad Gita expert. answer the question based on the context provided. If the context does not help, say 'I don't know'"),
    ("human", "query: {query} context: {context}"),
])

chain = (
    RunnablePassthrough() 
    | (lambda query: {"query": query, "context": "\n".join([doc.page_content for doc in vectorstore.similarity_search(query, k=3)])})
    | chat_prompt
    | llm
    | parser
)

chain.invoke("What is lust, how lust destroy humans??")


# there's no memeory implemented yet

'Based on the provided text from the Bhagavad Gita, I can say that lust (or kama) is considered the greatest enemy of living entities, and it induces them to remain entangled in the material world. Lust is described as a mode of passion that, if not elevated to the mode of goodness through spiritual attachment, leads to degradation into wrath.\n\nLust is said to be the symbol of ignorance, keeping the living entity within the material world. When living entities indulge in lustful activities, they may experience temporary happiness, but ultimately, it is an enemy that hinders their spiritual growth and understanding of their real position.\n\nThe text also mentions that lust cannot be satisfied by sense enjoyment, just like fire cannot be extinguished by a constant supply of fuel. In the material world, sex is considered the center of all activities, and this leads to the shackles of sex life, which are similar to prison bars for those who disobey the laws of the Lord.\n\nIn essence, l

### Using Langgraph

In [21]:
def retrieve_context(state):
    query = state["messages"][-1].content if state["messages"] else ""
    docs = vectorstore.similarity_search(query, k=3)
    context = "\n".join([doc.page_content for doc in docs])
    return {"context": context}

def response_generation(state):
    context = state.get("context", "")
    query = state["messages"][-1].content if state["messages"] else ""

    # Get the LLM's response
    response = llm.invoke(chat_prompt.format_messages(query=query, context=context, chat_history=state["messages"][:-1]))
    
    # Check the type of response and handle accordingly
    if hasattr(response, 'content'):
        response_text = response.content
    else:
        response_text = str(response)
        
    # Create and return AIMessage with the response text
    return {"messages": state["messages"] + [AIMessage(content=response_text)]}

# Handle commands like 'forget'
def handle_commands(state):
    last_message = state["messages"][-1].content.lower()
    
    if last_message.startswith("/forget"):
        # Clear the history except for this command
        return {"messages": [state["messages"][-1]], "command_executed": True}
    return {"command_executed": False}

In [22]:
# Complete implementation example
class State(TypedDict):
    messages: list
    context: str
    command_executed: bool

graph = StateGraph(State)

# Add all nodes with consistent names
graph.add_node("retrieve", retrieve_context)
graph.add_node("command_handler", handle_commands)
graph.add_node("respond", response_generation)

# Connect START to first node
graph.add_edge(START, "command_handler")

graph.add_conditional_edges(
    "command_handler",
    lambda state: state["command_executed"],
    {False: "retrieve", True: END}
)

graph.add_edge("retrieve", "respond")
graph.add_edge("respond", END)

workflow = graph.compile(checkpointer=MemorySaver())

In [24]:
state = {"messages": [], "context": "", "command_executed": False}

print("Chat with Bhagavad Gita Expert (type '/forget' to clear history, '/exit' to quit)")

while True:
    user_input = input("\nYou: ")
    
    if user_input.lower() == "/exit":
        break
        
    state["messages"] = add_messages(state["messages"], [HumanMessage(content=user_input)])
    
    state = workflow.invoke(
        state,
        config={
            "configurable": {
                "thread_id": "user_conversation_1",  
                "checkpoint_ns": "bhagavad_gita_chat" 
            }
        }
    )
    
    print(f"\nHuman: {state['messages'][-2].content}")
    print(f"AI: {state['messages'][-1].content}")

Chat with Bhagavad Gita Expert (type '/forget' to clear history, '/exit' to quit)


KeyboardInterrupt: Interrupted by user

# 2. multi-agent debate

create 2 agents (one optimistic, one pessimistic) and wire them in langgraph so they debate a user’s query. final node summarizes the debate for the user.

# 3. workflow assistant
input: user describes a task (e.g., “plan a study routine”).
graph:

- node 1 extracts subtasks

- node 2 searches web (or dummy data)

- node 3 synthesizes final plan

# 4. self-healing coder
a mini system where you feed code to one node, another node checks errors, and a third node proposes fixes. graph manages looping until code is “ok”.

# 5. daily journal summarizer
user dumps daily notes. one node organizes events, another node extracts key emotions, last node writes “insight of the day”. over time, graph keeps a memory chain.

# 6. task router
user says something, graph decides:

if it’s a factual q → send to retrieval node

if it’s casual chat → send to smalltalk node

if it’s a todo item → log into a simple json file.
perfect to practice conditional edges in langgraph.

# 7. multi-step reasoning tutor
user asks a math/logic q.
graph:

step 1 → rephrase problem

step 2 → propose reasoning steps

step 3 → verify with another node

step 4 → deliver clean final solution.